In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
import numpy as np
import csv

In [3]:
directors = []
actors = []
mpaaRatings = []
X = [] # This holds the movie data that the model will use to train.
y = [] # This holds whether the movies won an oscar or not that the model will use to train.
X_final_testset = [] # This holds movies from 2017 to 2019 that will be used on the website to display predictions.
y_final_testset = [] # This holds whether the movies from 2017 to 2019 won an oscar or not.

data = pd.read_csv("../Data/oscars.csv", encoding = "ISO-8859-1")
n = len(data.index)

for i in range(0, n):
    movie = list(data.iloc[i])
    year = movie[2]
    director = movie[7]
    actor1 = movie[11]
    actor2 = movie[15]
    actor3 = movie[19]
    mpaaRating = movie[23]
    
    if not(director in directors):
        directors.append(director)
    movie[7] = directors.index(director) # replace director to its unique numerical value
    movie[8] = int(movie[8])
    
    if not(actor1 in actors):
        actors.append(actor1)
    movie[11] = actors.index(actor1)
    movie[12] = int(movie[12])
    
    if not(actor2 in actors):
        actors.append(actor2)
    movie[15] = actors.index(actor2)
    movie[16] = int(movie[16])
    
    if not(actor3 in actors):
        actors.append(actor3)
    movie[19] = actors.index(actor3)
    movie[20] = int(movie[20])
    
    if not(mpaaRating in mpaaRatings):
        mpaaRatings.append(mpaaRating)
    movie[23] = mpaaRatings.index(mpaaRating) # replace mpaa rating to its unique numerical value
    
    if (year < 2017):
        X.append(movie[3:])
        y.append(int(movie[1]))
    else:
        X_final_testset.append(movie[3:])
        y_final_testset.append(int(movie[1]))
        

In [4]:
classifier = RandomForestClassifier(n_estimators=130, max_depth=27, criterion = "entropy", random_state=0, class_weight='balanced')
classifier.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='entropy', max_depth=27, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=130, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [5]:
predictions = classifier.predict(X_final_testset) # This predict function produces yes or no Oscar predictions.
pretty_list = "\n".join("{}\t\t{}".format(x, y) for x, y in zip(predictions, y_final_testset))
print("Predictions\tActual")
print(pretty_list)

Predictions	Actual
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		1
0		0
0		1
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		1
0		0
0		0
0		0
0		0


In [8]:
# This predict_proba function produces the probability of the movies winning an oscar.
proba_predictions = classifier.predict_proba(X_final_testset)[:,1]

% of 1 	Actual
0.12	0
0.15	0
0.08	0
0.35	0
0.28	0
0.07	0
0.07	0
0.42	1
0.12	0
0.09	1
0.23	0
0.48	0
0.13	0
0.23	0
0.32	0
0.13	0
0.05	0
0.31	0
0.08	0
0.12	0
0.11	0
0.44	1
0.15	0
0.12	0
0.21	0
0.17	0


In [9]:
pretty_list = "\n".join("{:.3f}\t{}".format(x, y) for x, y in zip(proba_predictions, y_final_testset))
print("% of 1 \tActual")
print(pretty_list)

% of 1 	Actual
0.123	0
0.146	0
0.085	0
0.354	0
0.285	0
0.069	0
0.069	0
0.415	1
0.115	0
0.092	1
0.231	0
0.485	0
0.131	0
0.231	0
0.323	0
0.131	0
0.046	0
0.308	0
0.085	0
0.123	0
0.108	0
0.438	1
0.154	0
0.115	0
0.208	0
0.169	0


In [10]:
data = []
data.append(["n_estimators", "max depth", "Round 1", "Round 2", "Round 3", "Round 4", "Round 5", "Round 6", "Round 7", "Round 8", "Round 9", "Round 10", "Avg F1 Score", "Avg Accuracy"])
n_values = [100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195, 200]
m_values = [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]

for i in range(0,21):
    for j in range(0,11):
        data_row = []
        data_row.append(n_values[i])
        data_row.append(m_values[j])
        f1_scores = []
        accuracy = []
        for k in range(0, 10):
            # train_test_split takes the datasets, shuffles the datasets, and splits it into training and testing sets.
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True, stratify = y)

            classifier = RandomForestClassifier(n_estimators=n_values[i], max_depth=m_values[j], criterion = "entropy", random_state=0, class_weight='balanced')
            classifier.fit(X_train,y_train)

            predictions = classifier.predict(X_test) # This predict function produces yes or no Oscar predictions.
            proba_predictions = classifier.predict_proba(X_test)[:,1] # This predict_proba function produces the probability of the movies winning an oscar.

            f1_scores.append(f1_score(y_test, predictions, average='weighted'))
            data_row.append(round(f1_score(y_test, predictions, average='weighted'),3))
            accuracy.append(accuracy_score(y_test,predictions))

        avg_f1 = np.mean(f1_scores)
        avg_accuracy = np.mean(accuracy)
        data_row.append(round(avg_f1,3))  
        data_row.append(round(avg_accuracy,3))
        data.append(data_row)
with open("random_forest_data_parametertuning3.csv","w+",newline='') as csv_file:
    csvWriter = csv.writer(csv_file,delimiter=',')
    csvWriter.writerows(data)

C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-p

C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-p

C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-p

C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-p

C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-p

...

C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-p

C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-p

C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-p

C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-p

C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [22]:
predictions = classifier.predict(X_test)
pretty_list = "\n".join("{}\t\t{}".format(x, y) for x, y in zip(predictions, y_test))
print("Predictions\tActual")
print(pretty_list)

Predictions	Actual
0		0
0		0
0		1
0		0
0		0
0		1
0		1
0		1
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0


In [23]:
# This predict_proba function produces the probability of the movies winning an oscar.
proba_predictions = classifier.predict_proba(X_test)[:,1]
pretty_list = "\n".join("{:.2f}\t{}".format(x, y) for x, y in zip(proba_predictions, y_test))
print("% of 1 \tActual")
print(pretty_list)

% of 1 	Actual
0.09	0
0.09	0
0.12	1
0.07	0
0.05	0
0.23	1
0.42	1
0.45	1
0.12	0
0.05	0
0.22	0
0.02	0
0.08	0
0.09	0
0.06	0
0.02	0
0.07	0
0.08	0
0.06	0
0.09	0
0.48	0
0.09	0
0.07	0
0.02	0
0.17	0


In [24]:
cm=confusion_matrix(y_test, predictions)
print(cm)
accuracy_score(y_test,predictions)

[[21  0]
 [ 4  0]]


0.84